In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import random
pd.set_option('display.max_colwidth', 100)

In [2]:
class_name = 'n'

# Validation Set
val_labels = pd.read_csv("../../radnlp_2024_train_val_20240731/en/main_task/val/label.csv")
val_path = "../prediction/ensemble/val/"
# r1
r1_val_1 = pd.read_csv(f"{val_path}deepseek-reasoner_val_01.csv")
r1_val_2 = pd.read_csv(f"{val_path}deepseek-reasoner_val_02.csv")
r1_val_3 = pd.read_csv(f"{val_path}deepseek-reasoner_val_03.csv")
# o1
o1_val_1 = pd.read_csv(f"{val_path}o1-mini_val_01.csv")
o1_val_2 = pd.read_csv(f"{val_path}o1-mini_val_02.csv")
o1_val_3 = pd.read_csv(f"{val_path}o1-mini_val_03.csv")

In [3]:
# Concatenate labels
labels = val_labels

# Concatenate predictions for r1 (deepseek-reasoner)
r1_pred_1 = r1_val_1
r1_pred_2 = r1_val_2
r1_pred_3 = r1_val_3

# Concatenate predictions for o1 (o1-mini)
o1_pred_1 = o1_val_1
o1_pred_2 = o1_val_2
o1_pred_3 = o1_val_3

In [4]:
data = pd.DataFrame({
    'id': labels['id'],
    f'r1_1_{class_name}': r1_pred_1[class_name],
    f'r1_2_{class_name}': r1_pred_2[class_name],
    f'r1_3_{class_name}': r1_pred_3[class_name],
    f'o1_1_{class_name}': o1_pred_1[class_name],
    f'o1_2_{class_name}': o1_pred_2[class_name],
    f'o1_3_{class_name}': o1_pred_3[class_name],
    'label': labels[class_name],
})
print(data)

          id r1_1_n r1_2_n r1_3_n o1_1_n o1_2_n o1_3_n label
0     147290     N0     N0     N0     N0     N0     N0    N0
1     241752     N0     N0     N0     N0     N0     N0    N0
2     876951     N0     N0     N0     N0     N0     N0    N0
3     923073     N0     N0     N0     N0     N0     N0    N0
4    1600422     N0     N0     N0     N0     N0     N0    N0
5    2318717     N0     N0     N0     N0     N0     N0    N0
6    2737981     N0     N0     N0     N0     N0     N0    N0
7    2755461     N2     N2     N2     N2     N2     N2    N2
8    3162670     N2     N2     N2     N2     N2     N2    N2
9    3462779     N2     N2     N2     N2     N2     N2    N2
10   4592263     N0     N0     N0     N0     N0     N0    N0
11   4644984     N2     N2     N2     N2     N2     N2    N2
12   4660316     N2     N2     N2     N2     N2     N2    N2
13   4724041     N0     N0     N0     N0     N0     N0    N0
14   4734929     N2     N2     N2     N2     N2     N2    N0
15   4924173     N0     

In [5]:
data.columns.tolist()[1:-1]

['r1_1_n', 'r1_2_n', 'r1_3_n', 'o1_1_n', 'o1_2_n', 'o1_3_n']

In [ ]:
import pandas as pd
from collections import defaultdict



# 1. Calculate model accuracies
model_columns = data.columns.tolist()[1:-1]
accuracies = {}

for model in model_columns:
    correct_predictions = (data[model] == data['label']).sum()
    total_predictions = len(data)
    accuracies[model] = correct_predictions / total_predictions

# 2. Create weighted voting system with tie-breakers
def enhanced_vote(row):
    vote_weights = defaultdict(float)
    
    # Add weights based on model accuracy
    for model in model_columns:
        class_val = row[model]
        vote_weights[class_val] += accuracies[model]
    
    # Get max weight and candidates
    max_weight = max(vote_weights.values())
    candidates = [k for k, v in vote_weights.items() if v == max_weight]
    
    # Enhanced tie-breaking: use class frequency as secondary criterion
    if len(candidates) > 1:
        class_counts = data['label'].value_counts().to_dict()
        return max(candidates, key=lambda x: class_counts.get(x, 0))
    
    return candidates[0]

# 3. Apply the enhanced voting
data['enhanced_prediction'] = data.apply(enhanced_vote, axis=1)

# 4. Calculate accuracy
accuracy = (data['enhanced_prediction'] == data['label']).mean()
print(f"Enhanced Accuracy: {accuracy:.2%}")
print("\nResults:")
print(data.head())

Enhanced Accuracy: 96.30%

Results:
        id r1_1_n r1_2_n r1_3_n o1_1_n o1_2_n o1_3_n label enhanced_prediction
0   147290     N0     N0     N0     N0     N0     N0    N0                  N0
1   241752     N0     N0     N0     N0     N0     N0    N0                  N0
2   876951     N0     N0     N0     N0     N0     N0    N0                  N0
3   923073     N0     N0     N0     N0     N0     N0    N0                  N0
4  1600422     N0     N0     N0     N0     N0     N0    N0                  N0


In [7]:
y_true = data['label']
y_pred = data['enhanced_prediction']

acc = accuracy_score(y_true, y_pred)
print(f"Accuracy: {acc:.4f}")

Accuracy: 0.9630


In [8]:
filtered_df = data[data['enhanced_prediction'] != data['label']]

filtered_df

,id,r1_1_n,r1_2_n,r1_3_n,o1_1_n,o1_2_n,o1_3_n,label,enhanced_prediction
14,4734929,N2,N2,N2,N2,N2,N2,N0,N2
36,11566958,N1,N1,N1,N1,N1,N1,N0,N1


In [9]:
accuracies

{'r1_1_n': 0.9444444444444444,
 'r1_2_n': 0.9259259259259259,
 'r1_3_n': 0.9444444444444444,
 'o1_1_n': 0.9259259259259259,
 'o1_2_n': 0.9444444444444444,
 'o1_3_n': 0.9444444444444444}